<a href="https://colab.research.google.com/github/AOdrio/Datathon/blob/main/bolaso%20-%20clasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
from matplotlib import pyplot as plt
import seaborn as sns 

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

SEED = 42

In [2]:
url='https://drive.google.com/file/d/1KH1fB83JDAWcMRCHKFf9SmcGH7MQkupE/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_original = pd.read_csv(url)

In [3]:
y = df_original.pop("ingreso")
X = df_original.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [4]:
print('Todo: ',df_original.shape)
print('Train:',X_train.shape)
print('Test: ',X_test.shape)

Todo:  (40000, 24)
Train: (32000, 24)
Test:  (8000, 24)


se remplazan los null por la media de la columna

In [5]:
edad_media = X_train['edad'].mean()

In [6]:
def transfromar_DataFrame (df,edad_media):
  df = df.copy()
  df = df.drop(df.columns[0],axis=1)
  #Remplazar nulls de la edad con la media de la columna
  df['edad'] = df['edad'].fillna(edad_media, inplace=False)

  #Transformar categóricas en numéricas
  df['peorcalificacionBCU'] = (df['peorcalificacionBCU'].
                              str.replace('5','6').
                              str.replace('4','5').
                              str.replace('3','4').
                              str.replace('2B','3').
                              str.replace('2A','2').
                              str.replace('1C','1').
                              str.replace('1A','0'))
  df['peorcalificacionBCU'] = pd.to_numeric(df['peorcalificacionBCU'], downcast='integer')
  return df

In [7]:
X_train = transfromar_DataFrame(X_train,edad_media)
X_test = transfromar_DataFrame(X_test,edad_media)

In [8]:
n_clases = 7

In [9]:
clases = pd.DataFrame(y_train)
clases['Quantile_rank'] = pd.qcut(clases['ingreso'], n_clases, labels = False)
clases['ingreso_medio'] = clases.groupby('Quantile_rank')['ingreso'].transform('mean')
valores_medios = clases.groupby(['Quantile_rank']).mean().drop('ingreso',axis=1)
MAPE_ingreso_medio = MAPE(clases['ingreso'],clases['ingreso_medio'])
print('MAPE linea base: ', round(MAPE_ingreso_medio *100, 2))

MAPE linea base:  9.52


In [10]:
y_train_2=clases['Quantile_rank']

In [58]:
rf_classifier = RandomForestClassifier(
                      min_samples_leaf=20,
                      n_estimators=1000,
                      bootstrap=True,
                      oob_score=True,
                      n_jobs=-1,
                      random_state=SEED,
                      max_features='auto')


In [59]:
rf_classifier.fit(X_train,y_train_2)
preds_rf = rf_classifier.predict(X_test)
pred_df = pd.DataFrame(preds_rf, columns=['Quantile_rank'])
MAPE_rf = MAPE(y_test,pd.merge(pred_df, valores_medios, on= 'Quantile_rank', how='left' )['ingreso_medio'])
print('MAPE: ', round(MAPE_rf *100, 2))

MAPE:  17.93


In [60]:
xgb_classifier = xgb.XGBClassifier(
            objective = "multi:softmax",
            num_class = n_clases,
            eval_metric = "merror",
            #tree_method = "gpu_hist",
            booster = "gbtree",
            random_state = SEED,
            n_estimators = 500,
            learning_rate = 0.01,
            max_depth = 9,
            min_child_weight = 7,
            subsample = 0.9,
            colsample_bytree = 1,
            colsample_bynode = 0.7,
            colsample_bylevel = 0.8,
            verbosity = 0,
            n_jobs = -1,
            )

In [61]:
xgb_classifier.fit(X_train,y_train_2)
preds_xgb = xgb_classifier.predict(X_test)
pred_xgb = pd.DataFrame(preds_xgb, columns=['Quantile_rank'])
MAPE_xgb = MAPE(y_test,pd.merge(pred_xgb, valores_medios, on= 'Quantile_rank', how='left' )['ingreso_medio'])
print('MAPE: ', round(MAPE_xgb *100, 2))


MAPE:  17.13
